In [58]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn import svm

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report



def predata(X,Y, norm=None, ts_size=0.2) -> list:

    xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=ts_size, random_state=89)
    # Normalisation
    if norm :
        scaler = norm()
        xtrain = scaler.fit_transform(xtrain)
        xtest = scaler.transform(xtest)

    return [xtrain, xtest, ytrain, ytest]



def testmodel(data:list, ml_model, specific_parameters:dict=None):
    
    # entrainement du modèle
    model = ml_model(**specific_parameters) if specific_parameters else ml_model()
    model.fit(data[0], data[2])

    # récupération des résultats avec le jeu de test, plus classique
    pred = model.predict(data[1])
    report = classification_report(data[3], pred, output_dict=True)

    return report



def comparemodels(X, Y, models:dict):

    res = []
    data = predata(X, Y, StandardScaler)

    for method, d in models.items():
        method_score = testmodel(data, d['name'], d['params'])
        met_res = [method]
        met_res.append(round(method_score['accuracy'], 2))
        met_res.append(round(method_score['0']['precision'], 2))
        met_res.append(round(method_score['0']['recall'], 2))
        met_res.append(round(method_score['0']['f1-score'], 2))
        res.append(met_res)

    res = pd.DataFrame(res, columns = ['method', 'accuracy', 'precision', 'recall', 'f1-score'])

    return res


#––––––––––––
# LANCEMENT
#––––––––––––


df = pd.read_csv('./titan_df.csv')
X = df.drop('survived', axis=1)
Y = df['survived']

methods = {
    'logistic': {
        'name': LogisticRegression,
        'params': {'max_iter':1000}
    },
    'svm': {
        'name': svm.SVC,
        'params': False
    },
    'perceptron': {
        'name': Perceptron,
        'params': False
    }
}

table = comparemodels(X, Y, methods)
table


,method,accuracy,precision,recall,f1-score
0,logistic,0.82,0.86,0.89,0.87
1,svm,0.80,0.83,0.90,0.86
2,perceptron,0.73,0.88,0.70,0.78
